In [10]:
import pandas as pd
import plotly.express as px

# Load once at the top
df = pd.read_csv("/Users/aschick/Downloads/timeline_data_4.csv")
df["year"] = df["start_year"]

cat_order = ["era", "migration", "civilization", "empire", "war", "religion"]
df["category"] = pd.Categorical(df["category"], cat_order, ordered=True)

def make_figure(start_year, end_year):
    # Overlap filter using start/end_year
    mask = (df["end_year"] >= start_year) & (df["start_year"] <= end_year)
    df_filtered = df[mask].copy()

    fig = px.scatter(
        df_filtered,
        x="year",
        y="category",
        color="category",
        hover_name="title",
        hover_data={
            "year": True,
            "continent": True,
            "start_year": True,
            "end_year": True,
            "description": True,
            "category": False,
        },
    )

    fig.update_traces(marker=dict(size=8, line=dict(width=1, color="black")))

    fig.update_xaxes(
        range=[start_year, end_year],
        title="Year (negative = BCE)",
        showgrid=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor="white",
    )

    fig.update_yaxes(
        title="Category",
        autorange="reversed",
    )

    fig.update_layout(
        template="plotly_dark",
        title=f"World History Mega-Timeline ({start_year} to {end_year})",
        hovermode="closest",
        dragmode="pan",
        legend_title_text="Category",
        height=600,
    )
    return fig


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

start_input = widgets.IntText(
    value=-5_000_000_000,
    description="Start year:",
    layout=widgets.Layout(width="300px")
)

end_input = widgets.IntText(
    value=2025,
    description="End year:",
    layout=widgets.Layout(width="300px")
)

button = widgets.Button(
    description="Update timeline",
    button_style="primary"
)

output = widgets.Output()

def on_click(b):
    with output:
        clear_output(wait=True)
        start_year = start_input.value
        end_year = end_input.value
        fig = make_figure(start_year, end_year)
        fig.show()

button.on_click(on_click)

display(widgets.HBox([start_input, end_input, button]), output)


Output()

In [13]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load once at the top
df = pd.read_csv("/Users/aschick/Downloads/timeline_data_4.csv")

# Use numeric year as the x value
# (assumes your CSV still has start_year / end_year as numbers)
df["x_year"] = df["start_year"]

cat_order = ["era", "migration", "civilization", "empire", "war", "religion"]
df["category"] = pd.Categorical(df["category"], cat_order, ordered=True)

def make_figure(start_year, end_year):
    # Overlap filter using start/end_year
    mask = (df["end_year"] >= start_year) & (df["start_year"] <= end_year)
    df_filtered = df[mask].copy()

    fig = px.scatter(
        df_filtered,
        x="x_year",
        y="category",
        color="category",
        hover_name="title",
        hover_data={
            "x_year": False,      # hide raw x
            "start_year": True,
            "end_year": True,
            "continent": True,
            "description": True,
            "category": False,
        },
    )

    fig.update_traces(marker=dict(size=8, line=dict(width=1, color="black")))

    fig.update_xaxes(
        range=[start_year, end_year],
        title="Year (negative = BCE, very negative = deep time)",
        showgrid=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor="white",
        type="linear",   # IMPORTANT: not 'date'
    )

    fig.update_yaxes(
        title="Category",
        autorange="reversed",
    )

    fig.update_layout(
        template="plotly_dark",
        title=f"World History Mega-Timeline ({start_year} to {end_year})",
        hovermode="closest",
        dragmode="pan",
        legend_title_text="Category",
        height=600,
    )
    return fig

# --- Widgets (keep using IntText for huge ranges) ---

start_input = widgets.IntText(
    value=-5_000_000_000,
    description="Start year:",
    layout=widgets.Layout(width="300px")
)

end_input = widgets.IntText(
    value=2025,
    description="End year:",
    layout=widgets.Layout(width="300px")
)

button = widgets.Button(
    description="Update timeline",
    button_style="primary"
)

output = widgets.Output()

def on_click(b):
    with output:
        clear_output(wait=True)
        start_year = start_input.value
        end_year = end_input.value
        fig = make_figure(start_year, end_year)
        fig.show()

button.on_click(on_click)

display(widgets.HBox([start_input, end_input, button]), output)


Output()

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------------------
# Load & prepare data
# ----------------------
df = pd.read_csv("/Users/aschick/Downloads/timeline_data_4.csv")

# Ensure year columns exist and are numeric
df["start_year"] = pd.to_numeric(df["start_year"])
df["end_year"] = pd.to_numeric(df["end_year"])

# Convenience column for numeric plotting
df["year"] = df["start_year"]

# Parse dates where possible; out-of-range or invalid => NaT
if "start_date" in df.columns:
    df["start_date"] = pd.to_datetime(df["start_date"], errors="coerce")
else:
    df["start_date"] = pd.NaT

if "end_date" in df.columns:
    df["end_date"] = pd.to_datetime(df["end_date"], errors="coerce")
else:
    df["end_date"] = pd.NaT

# Category order
cat_order = ["era", "migration", "civilization", "empire", "war", "religion"]
df["category"] = pd.Categorical(df["category"], cat_order, ordered=True)

# Only years >= this are eligible for true calendar dates
RECENT_MIN_YEAR = 1678  # pandas.Timestamp min year
RECENT_MAX_YEAR = 2262  # pandas.Timestamp max year


# ----------------------
# Figure builder
# ----------------------
def make_figure(start_year, end_year):
    """
    Build a dual-view timeline:
      - Row 1: numeric year axis (full deep-time range).
      - Row 2: real calendar dates for events that have valid start_date/end_date.
    """
    # Filter by overlap in numeric years
    mask = (df["end_year"] >= start_year) & (df["start_year"] <= end_year)
    df_filtered = df[mask].copy()

    # --- Subplot layout: 2 rows, shared categories on y ---
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_yaxes=True,
        row_heights=[0.6, 0.4],
        vertical_spacing=0.12,  # a bit more space between the two x-axes
        subplot_titles=(
            f"Numeric Year Timeline ({start_year} to {end_year})",
            "Recent History (Calendar Dates)",
        ),
    )

    # -------------------
    # Row 1: numeric years
    # -------------------
    fig_years = px.scatter(
        df_filtered,
        x="year",
        y="category",
        color="category",
        hover_name="title",
        hover_data={
            "year": True,
            "start_year": True,
            "end_year": True,
            "start_date": True,
            "end_date": True,
            "continent": True,
            "description": True,
            "category": False,
        },
    )

    for trace in fig_years.data:
        trace.marker.update(size=8, line=dict(width=1, color="black"))
        fig.add_trace(trace, row=1, col=1)

    # Top x-axis: keep title, hide tick labels to avoid overlap
    fig.update_xaxes(
        range=[start_year, end_year],
        title_text="Year (negative = BCE, very negative = deep time)",
        showgrid=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor="white",
        type="linear",
        showticklabels=False,   # <-- hide these
        row=1,
        col=1,
    )

    fig.update_yaxes(
        title_text="Category",
        autorange="reversed",
        row=1,
        col=1,
    )

    # -------------------
    # Row 2: real dates (recent)
    # -------------------
    df_recent = df_filtered[
        (df_filtered["start_date"].notna()) &
        (df_filtered["start_date"].dt.year >= RECENT_MIN_YEAR) &
        (df_filtered["start_date"].dt.year <= RECENT_MAX_YEAR)
    ].copy()

    if not df_recent.empty:
        fig_dates = px.scatter(
            df_recent,
            x="start_date",
            y="category",
            color="category",
            hover_name="title",
            hover_data={
                "start_date": True,
                "end_date": True,
                "start_year": True,
                "end_year": True,
                "continent": True,
                "description": True,
                "category": False,
            },
        )

        for trace in fig_dates.data:
            trace.showlegend = False  # legend already shown from top plot
            trace.marker.update(size=8, line=dict(width=1, color="black"))
            fig.add_trace(trace, row=2, col=1)

        # Set date-axis range based on requested year window & what's available
        min_recent_year = max(start_year, RECENT_MIN_YEAR)
        max_recent_year = min(
            end_year,
            int(df_recent["start_date"].dt.year.max())
        )

        if min_recent_year <= max_recent_year:
            recent_start = pd.to_datetime(f"{min_recent_year}-01-01")
            recent_end = pd.to_datetime(f"{max_recent_year}-12-31")
        else:
            recent_start = df_recent["start_date"].min()
            recent_end = df_recent["start_date"].max()

        fig.update_xaxes(
            range=[recent_start, recent_end],
            title_text="Calendar Date (recent events with precise dates)",
            showgrid=True,
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor="white",
            type="date",
            tickangle=-30,        # <-- tilt bottom labels a bit
            automargin=True,      # <-- let Plotly add room
            row=2,
            col=1,
        )

    else:
        # No precise dates in this window — add a note
        fig.add_annotation(
            text="",
            xref="x2 domain",
            yref="y2 domain",
            x=0.5,
            y=0.5,
            showarrow=False,
            font=dict(size=12),
        )
        fig.update_xaxes(
            title_text="Calendar Date (no precise-dated events here)",
            tickangle=-30,
            automargin=True,
            row=2,
            col=1,
        )

    # Hide duplicate category labels on lower y-axis (optional)
    fig.update_yaxes(showticklabels=False, row=2, col=1)

    fig.update_layout(
        template="plotly_dark",
        height=900,  # a bit taller
        hovermode="closest",
        dragmode="pan",
        legend_title_text="Category",
        title_text="World History Mega-Timeline (Years + Calendar Dates)",
        margin=dict(t=80, b=80, l=60, r=40),
    )

    return fig


# ----------------------
# Widgets
# ----------------------
start_input = widgets.IntText(
    value=-5_000_000_000,
    description="Start year:",
    layout=widgets.Layout(width="300px")
)

end_input = widgets.IntText(
    value=2025,
    description="End year:",
    layout=widgets.Layout(width="300px")
)

button = widgets.Button(
    description="Update timeline",
    button_style="primary"
)

output = widgets.Output()


def on_click(b):
    with output:
        clear_output(wait=True)
        start_year = start_input.value
        end_year = end_input.value
        fig = make_figure(start_year, end_year)
        fig.show()


button.on_click(on_click)

display(widgets.HBox([start_input, end_input, button]), output)


Output()

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------------------
# Load & prepare data
# ----------------------
df = pd.read_csv("/Users/aschick/Downloads/timeline_data_4.csv")

# Ensure year columns exist and are numeric
df["start_year"] = pd.to_numeric(df["start_year"])
df["end_year"] = pd.to_numeric(df["end_year"])

# Convenience column for numeric plotting
df["year"] = df["start_year"]

# Parse dates where possible; out-of-range or invalid => NaT
if "start_date" in df.columns:
    df["start_date"] = pd.to_datetime(df["start_date"], errors="coerce")
else:
    df["start_date"] = pd.NaT

if "end_date" in df.columns:
    df["end_date"] = pd.to_datetime(df["end_date"], errors="coerce")
else:
    df["end_date"] = pd.NaT

# Category order
cat_order = ["era", "migration", "civilization", "empire", "war", "religion"]
df["category"] = pd.Categorical(df["category"], cat_order, ordered=True)

# Only years >= this are eligible for true calendar dates
RECENT_MIN_YEAR = 1678  # pandas.Timestamp min year
RECENT_MAX_YEAR = 2262  # pandas.Timestamp max year


# ----------------------
# Figure builder
# ----------------------
def make_figure(start_year, end_year):
    """
    Build a dual-view timeline:
      - Row 1: numeric year axis (full deep-time range).
      - Row 2: real calendar dates for events that have valid start_date/end_date.
    """
    # Filter by overlap in numeric years
    mask = (df["end_year"] >= start_year) & (df["start_year"] <= end_year)
    df_filtered = df[mask].copy()

    # --- Subplot layout: 2 rows, shared categories on y ---
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_yaxes=True,
        row_heights=[0.6, 0.4],
        vertical_spacing=0.12,  # a bit more space between the two x-axes
        subplot_titles=(
            f"Numeric Year Timeline ({start_year} to {end_year})",
            "Recent History (Calendar Dates)",
        ),
    )

    # -------------------
    # Row 1: numeric years
    # -------------------
    fig_years = px.scatter(
        df_filtered,
        x="year",
        y="category",
        color="category",
        hover_name="title",
        hover_data={
            "year": True,
            "start_year": True,
            "end_year": True,
            "start_date": True,
            "end_date": True,
            "continent": True,
            "description": True,
            "category": False,
        },
    )

    for trace in fig_years.data:
        trace.marker.update(size=8, line=dict(width=1, color="black"))
        fig.add_trace(trace, row=1, col=1)

    # Top x-axis: SHOW tick labels, move them to top so they don’t collide
    fig.update_xaxes(
        range=[start_year, end_year],
        title_text="Year (negative = BCE, very negative = deep time)",
        showgrid=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor="white",
        type="linear",
        showticklabels=True,   # <-- was False
        side="top",            # <-- put labels above the top subplot
        tickangle=0,
        automargin=True,
        row=1,
        col=1,
    )

    fig.update_yaxes(
        title_text="Category",
        autorange="reversed",
        row=1,
        col=1,
    )

    # -------------------
    # Row 2: real dates (recent)
    # -------------------
    df_recent = df_filtered[
        (df_filtered["start_date"].notna()) &
        (df_filtered["start_date"].dt.year >= RECENT_MIN_YEAR) &
        (df_filtered["start_date"].dt.year <= RECENT_MAX_YEAR)
    ].copy()

    if not df_recent.empty:
        fig_dates = px.scatter(
            df_recent,
            x="start_date",
            y="category",
            color="category",
            hover_name="title",
            hover_data={
                "start_date": True,
                "end_date": True,
                "start_year": True,
                "end_year": True,
                "continent": True,
                "description": True,
                "category": False,
            },
        )

        for trace in fig_dates.data:
            trace.showlegend = False  # legend already shown from top plot
            trace.marker.update(size=8, line=dict(width=1, color="black"))
            fig.add_trace(trace, row=2, col=1)

        # Set date-axis range based on requested year window & what's available
        min_recent_year = max(start_year, RECENT_MIN_YEAR)
        max_recent_year = min(
            end_year,
            int(df_recent["start_date"].dt.year.max())
        )

        if min_recent_year <= max_recent_year:
            recent_start = pd.to_datetime(f"{min_recent_year}-01-01")
            recent_end = pd.to_datetime(f"{max_recent_year}-12-31")
        else:
            recent_start = df_recent["start_date"].min()
            recent_end = df_recent["start_date"].max()

        fig.update_xaxes(
            range=[recent_start, recent_end],
            title_text="Calendar Date (recent events with precise dates)",
            showgrid=True,
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor="white",
            type="date",
            tickangle=-30,        # <-- tilt bottom labels a bit
            automargin=True,      # <-- let Plotly add room
            row=2,
            col=1,
        )

    else:
        # No precise dates in this window — add a note
        fig.add_annotation(
            text="Calendar Date (no precise-dated events here)",
            xref="x2 domain",
            yref="y2 domain",
            x=0.5,
            y=0.5,
            showarrow=False,
            font=dict(size=12),
        )
        fig.update_xaxes(
            title_text="Calendar Date (no precise-dated events here)",
            tickangle=-30,
            automargin=True,
            row=2,
            col=1,
        )

    # Hide duplicate category labels on lower y-axis (optional)
    fig.update_yaxes(showticklabels=False, row=2, col=1)

    fig.update_layout(
        template="plotly_dark",
        height=900,  # a bit taller
        hovermode="closest",
        dragmode="pan",
        legend_title_text="Category",
        title_text="World History Mega-Timeline (Years + Calendar Dates)",
        margin=dict(t=100, b=80, l=60, r=40),  # more top margin for axis on top
    )

    return fig


# ----------------------
# Widgets
# ----------------------
start_input = widgets.IntText(
    value=-5_000_000_000,
    description="Start year:",
    layout=widgets.Layout(width="300px")
)

end_input = widgets.IntText(
    value=2025,
    description="End year:",
    layout=widgets.Layout(width="300px")
)

button = widgets.Button(
    description="Update timeline",
    button_style="primary"
)

output = widgets.Output()


def on_click(b):
    with output:
        clear_output(wait=True)
        start_year = start_input.value
        end_year = end_input.value
        fig = make_figure(start_year, end_year)
        fig.show()


button.on_click(on_click)

display(widgets.HBox([start_input, end_input, button]), output)


Output()